# Tests Models

In [1]:
import matplotlib.pyplot as plt
import ipywidgets as wdg
import numpy as np

#### Parsers

In [5]:
%matplotlib inline
def test_parser(upload):
    from models.parsers import Parsers
    
#     filename = list(upload['new'].keys())[0]
#     bitstream = upload['new'][filename]['content']
#     spectra, spectra_metadata = Parsers().single_txt(bitstream)
#     spectra, spectra_metadata = Parsers().single_csv(bitstream)
    formatted_upload = {key:value['content'] for key, value in upload['new'].items()}
    spectra, spectra_metadata = Parsers().multiple_txts(formatted_upload)
    
    
    test_name = list(spectra.keys())[0]
    print(spectra[test_name].class_id)

    for name in spectra.keys():
        plt.plot(spectra[name].energies,spectra[name].counts)
        
    plt.show()


upload_widget = wdg.FileUpload(description='Test parser',multiple=True)
upload_widget.observe(test_parser, names = 'value')
upload_widget

FileUpload(value={}, description='Test parser', multiple=True)

In [11]:
# upload_widget2 = wdg.FileUpload(description='Test parser',multiple=True)
# upload_widget2

In [12]:
# list_of_bits = upload_widget2.value['_12_03_2_labelled.txt']['content'].split(sep=None)
# list_lines = upload_widget2.value['HC Li LP40 operando_01.txt']['content'].split(sep=b'\n')
# list_lines[182] == b''
# for n, line in enumerate(list_lines):
#     if (not line.startswith(b'\t')) and (not line.startswith(b'#')):
#         print(n)
#         temporary_array = np.array(line.split(b'\t')).astype('float64')
# temporary_array

#### Spectrum

In [ ]:
from models.spectrum import Spectrum

def instantiate_spectrum():
    
    path = r'C:\Users\eibfl\Documents\Lead_projects\software_spectra_analysis\example_data_horiba_psi'
    filename = '_11_54_1.txt'
    w, c = np.loadtxt(path+'/'+filename).T
    return Spectrum(energies = w, counts= c, name='Li2CO3_D1_1t1h_g600_x50_20s3rep.txt')


def plot_spectrum(spectrum):
    
    print(spectrum.class_id)
    print(spectrum.metadata)
    
    plt.plot(spectrum.energies,spectrum.counts)
    plt.show()

    
example_raw_spectrum = instantiate_spectrum()
plot_spectrum(example_raw_spectrum)

#### Preprocessing

In [ ]:
def test_trim(spectrum, within=[350,1000]):
    from models.preprocessing import Preprocessing

    trimmed_spectrum = Preprocessing().trimming(spectrum, within)

    print(trimmed_spectrum.class_id)
    print(trimmed_spectrum.metadata)
    print(trimmed_spectrum.parent.class_id)
    print(trimmed_spectrum.parent.metadata)

    plt.plot(trimmed_spectrum.energies,trimmed_spectrum.counts)
    plt.show()
    
    return trimmed_spectrum 
    
example_trimmed_spectrum = test_trim(example_raw_spectrum)

#### Baseline correction

In [ ]:
def test_baseline(spectrum, log_p=-1.5, log_lambda = 7):
    from models.baselines import Baselines

    baseline_corrected_spectrum = Baselines().asymmetric_least_squares(spectrum, log_p, log_lambda)

    print(baseline_corrected_spectrum.class_id)
    print(baseline_corrected_spectrum.metadata)
    print(baseline_corrected_spectrum.parent.class_id)
    print(baseline_corrected_spectrum.parent.metadata)

    plt.plot(baseline_corrected_spectrum.energies,baseline_corrected_spectrum.counts)
    plt.show()
    
    return baseline_corrected_spectrum
    
example_baseline_spectrum = test_baseline(example_trimmed_spectrum)

#### Memory size of nested spectra
Each spectrum object has been output by a parent (input) spectrum, stored as the .parent atttribute. However, the object size does not grow with nesting, because each object is not storing a new object but making reference to an existing object in memory. Below, all spectra objects have the same space in memory, despite they being increasingly nested.

In [ ]:
# make recursive function to print spectrum tree
#estimated size in memory
import sys
print(example_raw_spectrum.metadata)
print('Memory size [bytes]: {}'.format(sys.getsizeof(example_raw_spectrum)))
print(example_trimmed_spectrum.metadata)
print('Memory size [bytes]: {}'.format(sys.getsizeof(example_trimmed_spectrum)))
print(example_baseline_spectrum.metadata)
print('Memory size [bytes]: {}'.format(sys.getsizeof(example_baseline_spectrum)))

#### Peak fitting

In [ ]:
def test_peakfit(spectrum, peak_bounds, guess_widths, lineshape):
    from models.fitpeaks import FitPeaks

    peak_fitted_spectrum = FitPeaks().fit_peaks(spectrum, peak_bounds, guess_widths, lineshape)

    print(peak_fitted_spectrum.class_id)
    print(peak_fitted_spectrum.metadata)
    print(peak_fitted_spectrum.parent.class_id)
    print(peak_fitted_spectrum.parent.metadata)

    plt.plot(spectrum.energies,spectrum.counts)
    plt.plot(peak_fitted_spectrum.energies,peak_fitted_spectrum.counts)
    for peak_number, peak in peak_fitted_spectrum.profiles.items():
        plt.plot(peak_fitted_spectrum.energies,peak)
        
    plt.show()
    
    return peak_fitted_spectrum

my_bounds = [(450,520),(530,610)]
my_guess_widhts = [50,50]
example_peakfit_spectrum = test_peakfit(example_baseline_spectrum, my_bounds, my_guess_widhts, lineshape='Pseudo-Voight-50Lor')